In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [2]:
relations = ['Cause-Effect', 'Component-Whole', 'Entity-Destination', 'Product-Producer', 'Entity-Origin',
             'Member-Collection', 'Message-Topic', 'Content-Container', 'Instrument-Agency', 'Other']

dataset_dir = os.path.join("..", "dataset", "")
train_path = os.path.join(dataset_dir, "train.txt")
max_words = 10000

texts = list()
labels = list()
with open(train_path) as fp:
    while True:
        line = fp.readline()
        label = fp.readline()
        if not line or not label:
            break
        texts.append(line.split(" ", 1)[1])
        labels.append(label.split('(')[0])

X = tf.constant(texts)
Y = list(map(relations.index, labels))
Y = np.eye(len(relations))[Y]

# X = X[0:100]
# Y = Y[0:100]

In [6]:
# text_inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
# preprocessor = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_preprocess/2")
# encoder_inputs = preprocessor(text_input)

preprocessor = hub.load("https://hub.tensorflow.google.cn/tensorflow/albert_en_preprocess/2")
# Step 1: tokenize batches of text inputs.
text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)] # This SavedModel accepts up to 2 text inputs.
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]
# Step 2 (optional): modify tokenized inputs.
pass
# Step 3: pack input sequences for the Transformer encoder.
seq_length = 40  # Your choice here.
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))  # Optional argument.
encoder_inputs = bert_pack_inputs(tokenized_inputs)

encoder = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_base/2", trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      
sequence_output = outputs["sequence_output"]

In [8]:
x = sequence_output
# x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, recurrent_dropout=0.2, dropout=0.2))(x)


x = layers.Bidirectional(layers.LSTM(128, recurrent_dropout=0.2, dropout=0.2))(x)

# x = layers.Reshape((128, -1, 1))(x)

# x2 = layers.Conv2D(1, (2, x.shape[2]), activation='relu')(x)
# x3 = layers.Conv2D(1, (3, x.shape[2]), activation='relu')(x)
# x4 = layers.Conv2D(1, (4, x.shape[2]), activation='relu')(x)
# x5 = layers.Conv2D(1, (5, x.shape[2]), activation='relu')(x)
# x6 = layers.Conv2D(1, (6, x.shape[2]), activation='relu')(x)
# x7 = layers.Conv2D(1, (7, x.shape[2]), activation='relu')(x)
# x8 = layers.Conv2D(1, (8, x.shape[2]), activation='relu')(x)
# x16 = layers.Conv2D(1, (16, x.shape[2]), activation='relu')(x)
# x32 = layers.Conv2D(1, (32, x.shape[2]), activation='relu')(x)
# x64 = layers.Conv2D(1, (64, x.shape[2]), activation='relu')(x)

# x2 = layers.MaxPooling2D(pool_size=(x2.shape[1], 1), padding='valid')(x2)
# x3 = layers.MaxPooling2D(pool_size=(x3.shape[1], 1), padding='valid')(x3)
# x4 = layers.MaxPooling2D(pool_size=(x4.shape[1], 1), padding='valid')(x4)
# x5 = layers.MaxPooling2D(pool_size=(x5.shape[1], 1), padding='valid')(x5)
# x6 = layers.MaxPooling2D(pool_size=(x6.shape[1], 1), padding='valid')(x6)
# x7 = layers.MaxPooling2D(pool_size=(x7.shape[1], 1), padding='valid')(x7)
# x8  = layers.MaxPooling2D(pool_size=(x8 .shape[1], 1), padding='valid')(x8 )
# x16 = layers.MaxPooling2D(pool_size=(x16.shape[1], 1), padding='valid')(x16)
# x32 = layers.MaxPooling2D(pool_size=(x32.shape[1], 1), padding='valid')(x32)
# x64 = layers.MaxPooling2D(pool_size=(x64.shape[1], 1), padding='valid')(x64)

# x2 = layers.Reshape((-1,))(x2)
# x3 = layers.Reshape((-1,))(x3)
# x4 = layers.Reshape((-1,))(x4)
# x5 = layers.Reshape((-1,))(x5)
# x6 = layers.Reshape((-1,))(x6)
# x7 = layers.Reshape((-1,))(x7)
# x8  = layers.Reshape((-1,))(x8 )
# x16 = layers.Reshape((-1,))(x16)
# x32 = layers.Reshape((-1,))(x32)
# x64 = layers.Reshape((-1,))(x64)

# [x2, x3, x4, x5, x6, x7, x8, x16, x32, x64]
# x = layers.Concatenate(axis=1)([x2, x3, x4,  x5, x6, x7])

outputs = layers.Dense(len(relations), activation='softmax')(x)
model = tf.keras.Model(inputs=text_inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
180/180 [==============================] - 98s 407ms/step - loss: 1.9697 - accuracy: 0.3059 - val_loss: 1.2585 - val_accuracy: 0.5844
Epoch 2/10
180/180 [==============================] - 72s 402ms/step - loss: 1.1613 - accuracy: 0.6036 - val_loss: 1.0937 - val_accuracy: 0.6250
Epoch 3/10
180/180 [==============================] - 74s 411ms/step - loss: 0.8783 - accuracy: 0.6981 - val_loss: 0.9790 - val_accuracy: 0.6500
Epoch 4/10
180/180 [==============================] - 71s 395ms/step - loss: 0.6759 - accuracy: 0.7717 - val_loss: 0.9753 - val_accuracy: 0.6469
Epoch 5/10
180/180 [==============================] - 72s 403ms/step - loss: 0.5516 - accuracy: 0.8144 - val_loss: 0.9664 - val_accuracy: 0.6547
Epoch 6/10
180/180 [==============================] - 72s 403ms/step - loss: 0.4388 - accuracy: 0.8505 - val_loss: 0.9452 - val_accuracy: 0.6953
Epoch 7/10
180/180 [==============================] - 71s 393ms/step - loss: 0.3093 - accuracy: 0.9043 - val_loss: 0.9740 - val_ac

In [5]:
test_path = os.path.join(dataset_dir, "test.txt")

test_texts = list()
with open(test_path) as fp:
    while True:
        line = fp.readline()
        if not line:
            break
        test_texts.append(line.split(" ", 1)[1])
        
X_test = tf.constant(test_texts)

preds = model.predict(X_test)
preds = np.argmax(preds,axis=1)
preds = list(map(lambda x:relations[x],preds))
pred_path = os.path.join('..','output', "prediction.txt")
with open(pred_path,'w+') as fp:
    fp.write("\n".join(preds))